In [12]:
import pandas as pd
import numpy as np
import json
import base64
import requests
from config.config import Client_ID,Client_Secret
#from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import spotipy.util as util
from requests import post , get


In [15]:
auth_string = f"{Client_ID}:{Client_Secret}"
auth_bytes = auth_string.encode("utf-8")
auth_base64 = base64.b64encode(auth_bytes).decode("utf-8")

url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': 'Basic ' + auth_base64
}
data = {
    'grant_type': 'client_credentials'
}

response = requests.post(url, headers=headers, data=data)
json_result = response.json()

if response.status_code == 200:
    token = json_result['access_token']
    print('Access Token: alındı')
else:
    print('Error:', response.status_code)
    print('Response:', json_result)

Access Token: alındı
